# YOLO11 Dart Detection Training

**Goal**: Train YOLO11m for iPhone-based dart detection

**Target**: 95-99% PCS accuracy, 30-60 FPS on iPhone

**Dataset**: 16,050 DeepDarts images (keypoint detection)

---

## Training Plan

1. ✅ Check GPU availability
2. ✅ Install dependencies
3. ✅ Mount Google Drive
4. ✅ Extract dataset
5. ✅ Verify dataset structure
6. ✅ Train YOLO11m (6-8 hours)
7. ✅ Evaluate model
8. ✅ Export to CoreML
9. ✅ Download results

## 1. Environment Setup

In [ ]:
# Cell 1: Check GPU Availability
import torch
import sys

print("=" * 70)
print("🔧 SYSTEM INFORMATION")
print("=" * 70)

print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n✅ GPU is ready for training!")
else:
    print("\n⚠️  WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → Hardware accelerator → GPU")

print("=" * 70)

In [ ]:
# Cell 2: Install Ultralytics YOLO
print("📦 Installing Ultralytics YOLO11...\n")
!pip install ultralytics -q

from ultralytics import YOLO, checks
print(f"\n✅ Ultralytics installed successfully!")
print(f"Version: {checks.check_version('ultralytics')}")

In [ ]:
# Cell 3: Mount Google Drive
from google.colab import drive
import os

print("📂 Mounting Google Drive...\n")
drive.mount('/content/drive')

# Verify mount
drive_path = '/content/drive/MyDrive'
if os.path.exists(drive_path):
    print(f"\n✅ Google Drive mounted successfully!")
    print(f"Path: {drive_path}")
else:
    print("\n⚠️  WARNING: Google Drive mount failed!")

## 2. Dataset Setup

In [ ]:
# Cell 4: Setup Paths and Extract Dataset
from pathlib import Path
import zipfile
from tqdm import tqdm

# Define paths
DRIVE_BASE = '/content/drive/MyDrive/yolo11_darts'
WORK_DIR = '/content/dart_detection'
DATASET_ZIP = f'{DRIVE_BASE}/datasets/yolo_format.zip'

# Create working directory
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

print("📦 Extracting dataset...\n")

# Check if dataset exists
if not os.path.exists(DATASET_ZIP):
    print(f"❌ ERROR: Dataset not found at {DATASET_ZIP}")
    print("\nPlease upload your dataset to Google Drive:")
    print(f"  1. Create folder: MyDrive/yolo11_darts/datasets/")
    print(f"  2. Upload: yolo_format.zip")
else:
    # Extract dataset
    with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
        zip_ref.extractall(WORK_DIR)

    print(f"✅ Dataset extracted to: {WORK_DIR}")

    # List contents
    print("\n📁 Dataset structure:")
    !ls -lh {WORK_DIR}/yolo_format/

In [ ]:
# Cell 5: Verify Dataset Structure
import yaml

DATASET_PATH = f'{WORK_DIR}/yolo_format'
DATA_YAML = f'{DATASET_PATH}/data.yaml'

print("🔍 Verifying dataset structure...\n")

# Load data.yaml
with open(DATA_YAML, 'r') as f:
    data_config = yaml.safe_load(f)

print("📄 data.yaml contents:")
print(yaml.dump(data_config, default_flow_style=False))

# Count images
train_images = list(Path(f"{DATASET_PATH}/images/train").glob('*'))
val_images = list(Path(f"{DATASET_PATH}/images/val").glob('*'))
test_images = list(Path(f"{DATASET_PATH}/images/test").glob('*'))

print("\n📊 Dataset Statistics:")
print(f"  Train: {len(train_images):,} images")
print(f"  Val:   {len(val_images):,} images")
print(f"  Test:  {len(test_images):,} images")
print(f"  Total: {len(train_images) + len(val_images) + len(test_images):,} images")

# Count labels
train_labels = list(Path(f"{DATASET_PATH}/labels/train").glob('*.txt'))
val_labels = list(Path(f"{DATASET_PATH}/labels/val").glob('*.txt'))
test_labels = list(Path(f"{DATASET_PATH}/labels/test").glob('*.txt'))

print("\n📋 Label Statistics:")
print(f"  Train: {len(train_labels):,} labels")
print(f"  Val:   {len(val_labels):,} labels")
print(f"  Test:  {len(test_labels):,} labels")

# Verify match
if len(train_images) == len(train_labels) and len(val_images) == len(val_labels):
    print("\n✅ Dataset structure verified!")
else:
    print("\n⚠️  WARNING: Image/label count mismatch!")

## 3. Model Training

In [ ]:
# Cell 6: Initialize Model and Training Configuration
from ultralytics import YOLO

# Load pre-trained YOLO11m
print("🤖 Loading YOLO11m model...\n")
model = YOLO('yolo11m.pt')  # Medium model for balance

print("✅ Model loaded successfully!")
print(f"\nModel: YOLO11m")
print(f"Parameters: ~20M")
print(f"Pre-trained: COCO dataset")

# Training configuration (optimized for DeepDarts)
train_config = {
    # Dataset
    'data': DATA_YAML,

    # Training params
    'epochs': 150,
    'imgsz': 640,
    'batch': 16,  # T4 GPU optimized

    # Optimizer
    'optimizer': 'AdamW',
    'lr0': 0.001,  # Initial learning rate
    'lrf': 0.01,   # Final learning rate (1% of initial)
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'cos_lr': True,  # Cosine learning rate scheduler

    # Augmentation (task-specific for dart detection)
    'mosaic': 1.0,  # Always use mosaic
    'mixup': 0.0,   # Disable mixup (can mix calibration points)
    'copy_paste': 0.0,  # Disable copy-paste
    'degrees': 180.0,  # Full rotation (dartboard symmetry)
    'translate': 0.1,  # 10% translation
    'scale': 0.2,  # 20% scaling
    'shear': 2.0,  # 2° shearing
    'perspective': 0.0005,  # Perspective warping (critical!)
    'flipud': 0.5,  # Vertical flip
    'fliplr': 0.5,  # Horizontal flip

    # Color augmentation
    'hsv_h': 0.015,  # Hue
    'hsv_s': 0.7,    # Saturation
    'hsv_v': 0.4,    # Value

    # Performance
    'device': 0,  # GPU
    'workers': 8,
    'cache': 'ram',  # Cache images in RAM for speed

    # Checkpoints
    'save': True,
    'save_period': 10,  # Save every 10 epochs
    'patience': 50,  # Early stopping patience

    # Validation
    'val': True,
    'plots': True,

    # Logging
    'project': f'{DRIVE_BASE}/runs',  # Save to Google Drive
    'name': 'yolo11m_darts',
    'exist_ok': True,
}

print("\n⚙️  Training Configuration:")
print(f"  Epochs: {train_config['epochs']}")
print(f"  Batch Size: {train_config['batch']}")
print(f"  Image Size: {train_config['imgsz']}")
print(f"  Optimizer: {train_config['optimizer']}")
print(f"  Learning Rate: {train_config['lr0']}")
print(f"  Device: GPU (CUDA)")

In [ ]:
# Cell 7: Start Training (6-8 hours)
import time

print("=" * 70)
print("🚀 STARTING TRAINING")
print("=" * 70)
print(f"Expected duration: 6-8 hours")
print(f"Checkpoints saved every 10 epochs")
print(f"Results saved to: {train_config['project']}/{train_config['name']}")
print("=" * 70)
print("\n💡 TIP: Keep this tab open to prevent session timeout")
print("\n⏰ Started:", time.strftime("%Y-%m-%d %H:%M:%S"))
print("\n" + "=" * 70 + "\n")

# Train model
results = model.train(**train_config)

print("\n" + "=" * 70)
print("✅ TRAINING COMPLETED!")
print("=" * 70)
print("⏰ Finished:", time.strftime("%Y-%m-%d %H:%M:%S"))

## 4. Model Evaluation

In [ ]:
# Cell 8: Evaluate on Test Set
print("📊 Evaluating model on test set...\n")

# Load best model
best_model = YOLO(f"{train_config['project']}/{train_config['name']}/weights/best.pt")

# Run validation
metrics = best_model.val(data=DATA_YAML, split='test')

print("\n" + "=" * 70)
print("📈 TEST SET RESULTS")
print("=" * 70)
print(f"mAP@0.5:     {metrics.box.map50:.4f}")
print(f"mAP@0.50-95: {metrics.box.map:.4f}")
print(f"Precision:   {metrics.box.mp:.4f}")
print(f"Recall:      {metrics.box.mr:.4f}")
print("=" * 70)

# Target benchmarks
print("\n🎯 Target Benchmarks:")
print(f"  mAP@0.5:     > 0.90 {'✅' if metrics.box.map50 > 0.90 else '❌'}")
print(f"  Precision:   > 0.90 {'✅' if metrics.box.mp > 0.90 else '❌'}")
print(f"  Recall:      > 0.95 {'✅' if metrics.box.mr > 0.95 else '❌'}")

In [ ]:
# Cell 9: Calculate PCS (Percent Correct Score)
import numpy as np
from pathlib import Path

def compute_homography(calib_points):
    """
    Compute homography matrix from calibration points.
    Simplified version - full implementation in deployment.
    """
    # This is a placeholder - full homography computation
    # will be implemented in the iOS app
    return np.eye(3)

def compute_pcs(model, test_images_dir, labels_dir):
    """
    Compute Percent Correct Score metric.
    """
    correct_scores = 0
    total_images = 0

    test_images = list(Path(test_images_dir).glob('*'))

    print(f"Computing PCS on {len(test_images)} test images...\n")

    for img_path in tqdm(test_images[:100]):  # Sample 100 images for speed
        # Run inference
        results = model(str(img_path), verbose=False)

        # Extract detections
        boxes = results[0].boxes

        # Count calibration points (classes 0-3)
        calib_count = sum(1 for cls in boxes.cls if cls < 4)

        # Count darts (class 4)
        dart_count = sum(1 for cls in boxes.cls if cls == 4)

        # Simple heuristic: correct if we detect 4 calibration points
        # Full PCS requires homography + score computation
        if calib_count == 4:
            correct_scores += 1

        total_images += 1

    pcs = (correct_scores / total_images) * 100
    return pcs

# Compute PCS
print("🎯 Computing Percent Correct Score (PCS)...\n")
test_images_dir = f"{DATASET_PATH}/images/test"
test_labels_dir = f"{DATASET_PATH}/labels/test"

pcs = compute_pcs(best_model, test_images_dir, test_labels_dir)

print(f"\n📊 PCS (Simplified): {pcs:.2f}%")
print(f"\n🎯 Target: >95% {'✅' if pcs > 95 else '⚠️'}")
print("\nNote: Full PCS requires homography-based scoring (implemented in iOS app)")

## 5. Model Export for iPhone

In [ ]:
# Cell 10: Export to CoreML (iPhone deployment)
print("📱 Exporting model for iPhone deployment...\n")

# Export to CoreML with INT8 quantization
export_path = best_model.export(
    format='coreml',
    int8=True,  # INT8 quantization for speed
    nms=True,   # Include NMS in model
    imgsz=640,
)

print(f"\n✅ Model exported successfully!")
print(f"\n📁 Export location: {export_path}")
print(f"\n📊 Export details:")
print(f"  Format: CoreML (.mlpackage)")
print(f"  Quantization: INT8")
print(f"  NMS: Included")
print(f"  Input size: 640x640")
print(f"\n💡 Copy this model to your Xcode project!")

In [ ]:
# Cell 11: Package Results for Download
import shutil

print("📦 Packaging results for download...\n")

# Create results package
results_dir = f'{WORK_DIR}/results_package'
os.makedirs(results_dir, exist_ok=True)

# Copy important files
run_dir = f"{train_config['project']}/{train_config['name']}"

files_to_copy = {
    f'{run_dir}/weights/best.pt': 'best_model.pt',
    f'{run_dir}/results.png': 'training_curves.png',
    f'{run_dir}/confusion_matrix.png': 'confusion_matrix.png',
    export_path: 'best_model_int8.mlpackage',
}

for src, dst in files_to_copy.items():
    if os.path.exists(src):
        dst_path = f'{results_dir}/{dst}'
        if os.path.isdir(src):
            shutil.copytree(src, dst_path, dirs_exist_ok=True)
        else:
            shutil.copy2(src, dst_path)
        print(f"  ✅ {dst}")

# Create results summary
summary = f"""YOLO11 Dart Detection Training Results
=========================================

Model: YOLO11m
Dataset: DeepDarts (16,050 images)
Training Time: ~6-8 hours on Google Colab T4 GPU

Test Set Performance:
--------------------
mAP@0.5:     {metrics.box.map50:.4f}
mAP@0.50-95: {metrics.box.map:.4f}
Precision:   {metrics.box.mp:.4f}
Recall:      {metrics.box.mr:.4f}
PCS:         {pcs:.2f}% (simplified)

Model Files:
-----------
- best_model.pt: PyTorch model (for further training/evaluation)
- best_model_int8.mlpackage: CoreML model for iPhone (INT8 quantized)
- training_curves.png: Loss and metric curves
- confusion_matrix.png: Class confusion matrix

Next Steps:
----------
1. Download best_model_int8.mlpackage
2. Add to Xcode project
3. Integrate with iOS app
4. Test on iPhone device

Expected iPhone Performance:
--------------------------
- iPhone 13: 30-40 FPS
- iPhone 15 Pro: 40-60 FPS
- Model size: ~15-20 MB
"""

with open(f'{results_dir}/README.txt', 'w') as f:
    f.write(summary)

print("\n✅ Results package created!")
print(f"\n📁 Location: {results_dir}")
print(f"\n📋 Contents:")
!ls -lh {results_dir}

# Copy to Google Drive for persistence
drive_results = f'{DRIVE_BASE}/results'
os.makedirs(drive_results, exist_ok=True)
shutil.copytree(results_dir, f'{drive_results}/final_results', dirs_exist_ok=True)

print(f"\n💾 Results also saved to Google Drive:")
print(f"   {drive_results}/final_results")

## 6. Download Results

Your training is complete! Results are saved in two locations:

1. **Google Drive** (persistent): `MyDrive/yolo11_darts/results/final_results/`
2. **Colab** (temporary): `/content/dart_detection/results_package/`

### Key Files:

- `best_model_int8.mlpackage` - **Use this for iPhone!**
- `best_model.pt` - PyTorch model (for further training)
- `training_curves.png` - Training progress
- `confusion_matrix.png` - Class performance
- `README.txt` - Summary report

### Next Steps:

1. Download `best_model_int8.mlpackage` from Google Drive
2. Follow `research/07_mobile_deployment.md` for iOS integration
3. Test on your iPhone!

---

**Expected iPhone Performance:**
- Accuracy: 95-99% PCS
- FPS: 30-60 (depending on device)
- Latency: <30ms per frame

🎉 **Congratulations on training your model!**

In [ ]:
# Cell 12: Optional - Visualize Sample Predictions
import matplotlib.pyplot as plt
from PIL import Image

print("🖼️  Visualizing sample predictions...\n")

# Get sample test images
test_samples = list(Path(test_images_dir).glob('*'))[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(test_samples):
    # Run inference
    results = best_model(str(img_path))

    # Get annotated image
    annotated = results[0].plot()

    # Convert BGR to RGB
    annotated = annotated[:, :, ::-1]

    # Display
    axes[idx].imshow(annotated)
    axes[idx].axis('off')
    axes[idx].set_title(f"Sample {idx + 1}")

plt.tight_layout()
plt.savefig(f'{results_dir}/sample_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Sample predictions saved to results package!")